# Parsing score with music21
* Goal: parse a score and put all notes involved into a python set

In [12]:
from music21 import *
import numpy as np
import math
import os
import time

In [13]:
score = converter.parse('bourree.xml')

In [14]:
score.analyze('key')

<music21.key.Key of G major>

## Finding number of Measures in score


In [15]:
print(score.parts)
print(len(score.parts))
print(f'there are {len(score.parts)} part(s) in score.parts')
for item in score.parts:
    print(item)
# for i in range(len(score.parts)):
#     print(score.parts[i]) 

<music21.stream.iterator.StreamIterator for Score:0x7f84da6d4810 @:0>
1
there are 1 part(s) in score.parts
<music21.stream.Part Part 1>


In [16]:
# isolate out the only part (聲部)
part = score.parts[0]
print(f'there are {len(part)} element(s) in part')
for i,item in enumerate(part, 1):
    print(f'item {i} : {item}') 

there are 24 element(s) in part
item 1 : P1: Part 1: 
item 2 : <music21.stream.Measure 1 offset=0.0>
item 3 : <music21.spanner.Slur <music21.note.Note D><music21.note.Note B>>
item 4 : <music21.spanner.Slur <music21.note.Note B><music21.note.Note B>>
item 5 : <music21.dynamics.Crescendo <music21.note.Note E><music21.note.Note G>>
item 6 : <music21.spanner.Slur <music21.note.Note E><music21.note.Note D>>
item 7 : <music21.spanner.Slur <music21.note.Note C><music21.note.Note B>>
item 8 : <music21.spanner.Slur <music21.note.Note B><music21.note.Note G>>
item 9 : <music21.dynamics.Crescendo <music21.note.Note B><music21.note.Note G>>
item 10 : <music21.spanner.Slur <music21.note.Note A><music21.note.Note C#>>
item 11 : <music21.spanner.Slur <music21.note.Note A><music21.note.Note C#>>
item 12 : <music21.dynamics.Diminuendo <music21.note.Note D><music21.note.Rest 16th>>
item 13 : <music21.spanner.Slur <music21.note.Note D><music21.note.Note B>>
item 14 : <music21.spanner.Slur <music21.note.

In [17]:
"""playing with Isaac's Code: verified is correct, but how does the math work?"""
# score: a stream object that can contain multiple streams
beats = score.getTimeSignatures(recurse=False)[0].denominator       #retrieve the time signature of object score and save only the denominator (duration of each beat)
num_measures = int(math.ceil(score.highestTime / beats))            #find the when the last note finishes playing in object stream, divide it by duration of each beat, and take the ceiling
print(f'there are {num_measures} measures in score')

there are 10 measures in score


## Collect all notes from all measures in a score

### printing all notes from all measures (attempt1)

In [18]:
# measures start with index 1 and can be accessed from score.measure(measure_num)

for i in range(1, num_measures+1, 1):
    measure_notes = part.measure(i).notes       #note in this example there is only one part, so this is working
    print(f'measure {i}:')
    for note in measure_notes:
        print(f'\t{note}')

measure 1:
	<music21.note.Note D>
measure 2:
	<music21.note.Note D>
	<music21.note.Note B>
	<music21.note.Note C>
	<music21.note.Note B>
	<music21.note.Note A>
	<music21.note.Note G>
measure 3:
	<music21.note.Note E>
	<music21.note.Note G>
	<music21.note.Note F#>
	<music21.note.Note E>
measure 4:
	<music21.note.Note D>
	<music21.note.Note C>
	<music21.note.Note B>
	<music21.note.Note A>
	<music21.note.Note B>
	<music21.note.Note C>
	<music21.note.Note A>
measure 5:
	<music21.note.Note B>
	<music21.note.Note G>
	<music21.note.Note A>
measure 6:
	<music21.note.Note B>
	<music21.note.Note C#>
	<music21.note.Note D>
	<music21.note.Note B>
	<music21.note.Note C#>
	<music21.note.Note D>
	<music21.note.Note E>
	<music21.note.Note C#>
measure 7:
	<music21.note.Note D>
	<music21.note.Note E>
	<music21.note.Note F#>
	<music21.note.Note D>
	<music21.note.Note E>
	<music21.note.Note F#>
	<music21.note.Note G>
	<music21.note.Note E>
measure 8:
	<music21.note.Note F#>
	<music21.note.Note G>
	<music2

## Collecting all notes from a measure and put them into a set

In [19]:
"""
    input:  measure (music21.stream.Measure)
    output: (set(music21.note.Note)): a set containing all notes present in the mxml file
"""
def collect_all_notes_from_measure(measure):
    return frozenset(note for note in measure.notes)

> compute the number of measures from a score

In [20]:
"""Isacc's code refactored into a function: but still do not know how does the math works"""
def find_num_measures_in_score(score):
    #might have to iterate through this part if there are multiple time signatures in a given score
    # beats = [timeSig.denominator for timeSig in score.getTimeSignatures(recurse=False)]   #the multiple time sig version
    beats = score.getTimeSignatures(recurse=False)[0].denominator       
    num_measures = int(math.ceil(score.highestTime / beats))
    return num_measures

> given score, get all notes from all parts which each contains different measures

In [25]:
"""
input: score (music21.stream.Score)
output: all_notes (set(music21.stream.Note))
"""
def collect_all_notes_from_score(score):
    all_notes = set()
    for part in score.parts:
        num_measures = find_num_measures_in_score(part)
        for i in range(1, num_measures+1, 1):
            all_notes.add(collect_all_notes_from_measure(part.measure(i)))
    return all_notes

> flattenning set

In [26]:
def flatten_frozenset_in_sets(input_set:set, final:set) -> set:
    for elem in input_set:
        if type(elem) == frozenset:
            flatten_frozenset_in_sets(set(elem), final)
        else:
            final.add(elem)

## Driver/test code for note collection from a given score

In [27]:
notes_set = collect_all_notes_from_score(score)
print(len(notes_set))
print(notes_set)
flattend_set = set()
flatten_frozenset_in_sets(notes_set, flattend_set)
print(flattend_set)         #"issue": why are there duplicate notes? are they different in pitch/dynamics/...etc.?
assert len(flattend_set) != len(notes_set), "flattend set should exclude repetitive notes in original set"

10
{frozenset({<music21.note.Note B>, <music21.note.Note B>, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note A>}), frozenset({<music21.note.Note D>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note G>, <music21.note.Note B>, <music21.note.Note B>}), frozenset({<music21.note.Note B>, <music21.note.Note G>, <music21.note.Note A>}), frozenset({<music21.note.Note F#>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note F#>, <music21.note.Note E>, <music21.note.Note G>, <music21.note.Note E>}), frozenset({<music21.note.Note B>, <music21.note.Note C>, <music21.note.Note G>, <music21.note.Note D>, <music21.note.Note A>, <music21.note.Note B>}), frozenset({<music21.note.Note E>, <music21.note.Note F#>, <music21.note.Note G>, <music21.note.Note E>}), frozenset({<music21.note.Note C#>, <music21.note.Note B>, <music21.note.Note D>, <music21.note.Note C#>, <music21.note.Note D>, <musi

# Notes:
* score: either .mxml or pdf(?)
* audio: assume a numpy array 